In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

In [6]:
input_size = 784
no_classes = 10
batch_size = 100
total_batches = 200

In [7]:
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

In [24]:
def add_variable_summary(tf_variable, summary_name):
    with tf.name_scope(summary_name + '_summary'):
        mean = tf.reduce_mean(tf_variable)
        tf.summary.scalar('Mena', mean)
        with tf.name_scope('standard_deviation'):
            standard_deviation = tf.sqrt(tf.reduce_mean(tf.sqrt(tf_variable - mean)))
        tf.summary.scalar('StandardDeviation', standard_deviation)
        tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
        tf.summary.scalar('Minium', tf.reduce_mean(tf_variable))
        tf.summary.histogram('Histogram', tf_variable)

In [25]:
x_input_reshape = tf.reshape(x_input, [-1, 28, 28, 1], name='input_reshape')

In [26]:
x_input_reshape.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

In [27]:
def convalution_layer(input_layer, filters, kernel_size=[3,3], activation=tf.nn.relu):
    layer = tf.layers.conv2d(inputs=input_layer, 
                             filters=filters, 
                             kernel_size=kernel_size,
                             activation=activation)
    add_variable_summary(layer, 'convolution')
    return layer

In [28]:
def pooling_layer(input_layer, pool_size=[2, 2], strides=2):
    layer = tf.layers.max_pooling2d(inputs=input_layer,
                                   pool_size=pool_size,
                                   strides=strides)
    add_variable_summary(layer, 'pooling')
    return layer

In [29]:
def dense_layer(input_layer, units, activation=tf.nn.relu):
    layer = tf.layers.dense(inputs=input_layer,
                            units=units,
                            activation=activation)
    add_variable_summary(layer, 'dense')
    return layer

In [30]:
convlution_layer_1 = convalution_layer(x_input_reshape, 64)
pooling_layer_1 = pooling_layer(convlution_layer_1)
convalution_layer_2 = convalution_layer(pooling_layer_1, 128)
pooling_layer_2 = pooling_layer(convalution_layer_2)

flattend_pool = tf.reshape(pooling_layer_2, [-1, 5 * 5 * 128], name='flattend_pool')
dense_layer_bottleneck = dense_layer(flattend_pool, 1024)

Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [31]:
dropout_bool = tf.placeholder(tf.bool)
dropout_layer = tf.layers.dropout(inputs=dense_layer_bottleneck,
                                 rate=0.4,
                                 training=dropout_bool)

Instructions for updating:
Use keras.layers.dropout instead.


In [32]:
logits = dense_layer(dropout_layer, no_classes)

In [34]:
with tf.name_scope('loss'):
    softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_input, logits=logits)
    loss_operation = tf.reduce_mean(softmax_cross_entropy, name='loss')
    tf.summary.scalar('loss', loss_operation)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [35]:
with tf.name_scope('optimiser'):
    optimiser = tf.train.AdamOptimizer().minimize(loss_operation)

In [36]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        predictions = tf.argmax(logits, 1)
        correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
    with tf.name_scope('accuracy'):
        accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [37]:
tf.summary.scalar('accuracy', accuracy_operation)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [39]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [40]:
merged_summary_operation = tf.summary.merge_all()
train_summary_writer = tf.summary.FileWriter('./train', session.graph)
test_summary_writer = tf.summary.FileWriter('./test')

In [41]:
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels

In [43]:
for batch_no in range(total_batches):
    mnist_batch = mnist_data.train.next_batch(batch_size)
    train_images, train_labels = mnist_batch[0], mnist_batch[1]
    _, merged_summary = session.run([optimiser, merged_summary_operation],
                                   feed_dict={
                                       x_input: train_images,
                                       y_input: train_labels,
                                       dropout_bool: True
                                   })
    train_summary_writer.add_summary(merged_summary, batch_no)
    if batch_no % 10 == 0:
        merged_summary, _ = session.run([merged_summary_operation, accuracy_operation],
                                       feed_dict={
                                           x_input: test_images,
                                           y_input: test_labels,
                                           dropout_bool: False
                                       })
        test_summary_writer.add_summary(merged_summary, batch_no)

In [44]:
!tensorboard --logdir=./train

^C
